In [15]:
import numpy as np
import torch
import time

from radiomics import cMatrices              # C 扩展
from radiomics import cmatrices as pycm      # 你的 numpy / torch 实现

In [8]:
def compare_one_case(
    bounding_box_size,
    kernel_radius,
    force2D=False,
    force2Ddimension=0,
    bidirectional=True,
):
    """
    运行一个参数配置，比较：
      - cMatrices.generate_angles（C 版）
      - cmatrices.generate_angles_np（NumPy 版）
      - cmatrices.generate_angles_torch（Torch 版）
    的结果是否完全一致。
    """
    bounding_box_size = np.asarray(bounding_box_size, dtype=np.int32)
    distances = np.arange(1, kernel_radius + 1, dtype=np.int32)

    # --- C 扩展版本 ---
    angles_c = cMatrices.generate_angles(
        bounding_box_size,
        distances,
        bidirectional,
        force2D,
        force2Ddimension,
    )
    angles_c = np.asarray(angles_c, dtype=np.int32)

    # --- NumPy 版本 ---
    angles_np = pycm.generate_angles_np(
        bounding_box_size,
        distances,
        bidirectional=bidirectional,
        force2D=force2D,
        force2Ddimension=force2Ddimension,
    ).astype(np.int32)

    # --- Torch 版本 ---
    angles_torch = pycm.generate_angles_torch(
        bounding_box_size,
        distances,
        bidirectional=bidirectional,
        force2D=force2D,
        force2Ddimension=force2Ddimension,
        dtype=torch.int32,
        device="cpu",
    )
    angles_torch_np = angles_torch.cpu().numpy().astype(np.int32)

    # --- 比较形状 ---
    print("C shape     :", angles_c.shape)
    print("NumPy shape :", angles_np.shape)
    print("Torch shape :", angles_torch_np.shape)

    if angles_c.shape != angles_np.shape or angles_c.shape != angles_torch_np.shape:
        raise AssertionError(
            f"Shape mismatch: C={angles_c.shape}, np={angles_np.shape}, torch={angles_torch_np.shape}"
        )

    # --- 比较内容 ---
    same_np = np.array_equal(angles_c, angles_np)
    same_torch = np.array_equal(angles_c, angles_torch_np)

    print("C vs NumPy equal :", same_np)
    print("C vs Torch equal :", same_torch)

    if not same_np:
        print(">>> C 和 NumPy 不一致，前几行对比：")
        print("C[0:10]:\n", angles_c[:10])
        print("NP[0:10]:\n", angles_np[:10])

    if not same_torch:
        print(">>> C 和 Torch 不一致，前几行对比：")
        print("C[0:10]:\n", angles_c[:10])
        print("Torch[0:10]:\n", angles_torch_np[:10])

    if same_np and same_torch:
        print(
            f"✅ OK: size={bounding_box_size.tolist()}, kernel_radius={kernel_radius}, "
            f"force2D={force2D}, force2Ddim={force2Ddimension}, bidirectional={bidirectional}, "
            f"Na={angles_c.shape[0]}"
        )

In [11]:
rng = np.random.default_rng(0)

for i in range(10):
    Nd = int(rng.integers(2, 4))  # 2D 或 3D
    size = rng.integers(3, 10, size=Nd)
    kernel_radius = int(rng.integers(1, 4))
    force2D = bool(rng.integers(0, 2))
    force2Ddim = int(rng.integers(0, Nd)) if force2D else 0
    bidirectional = bool(rng.integers(0, 2))

    print(f"\n=== Random case {i} ===")
    compare_one_case(
        bounding_box_size=size,
        kernel_radius=kernel_radius,
        force2D=force2D,
        force2Ddimension=force2Ddim,
        bidirectional=bidirectional,
    )


=== Random case 0 ===
C shape     : (13, 3)
NumPy shape : (13, 3)
Torch shape : (13, 3)
C vs NumPy equal : True
C vs Torch equal : True
✅ OK: size=[7, 6, 4], kernel_radius=1, force2D=False, force2Ddim=0, bidirectional=False, Na=13

=== Random case 1 ===
C shape     : (4, 2)
NumPy shape : (4, 2)
Torch shape : (4, 2)
C vs NumPy equal : True
C vs Torch equal : True
✅ OK: size=[4, 8], kernel_radius=2, force2D=True, force2Ddim=1, bidirectional=True, Na=4

=== Random case 2 ===
C shape     : (24, 3)
NumPy shape : (24, 3)
Torch shape : (24, 3)
C vs NumPy equal : True
C vs Torch equal : True
✅ OK: size=[8, 7, 6], kernel_radius=2, force2D=True, force2Ddim=0, bidirectional=True, Na=24

=== Random case 3 ===
C shape     : (124, 3)
NumPy shape : (124, 3)
Torch shape : (124, 3)
C vs NumPy equal : True
C vs Torch equal : True
✅ OK: size=[3, 5, 9], kernel_radius=2, force2D=False, force2Ddim=0, bidirectional=True, Na=124

=== Random case 4 ===
C shape     : (244, 3)
NumPy shape : (244, 3)
Torch shape

In [12]:
def check_equal_once(
    bounding_box_size,
    kernel_radius,
    force2D=False,
    force2Ddimension=0,
    bidirectional=True,
):
    bounding_box_size = np.asarray(bounding_box_size, dtype=np.int32)
    distances = np.arange(1, kernel_radius + 1, dtype=np.int32)

    # C 版本
    angles_c = cMatrices.generate_angles(
        bounding_box_size,
        distances,
        bidirectional,
        force2D,
        force2Ddimension,
    )
    angles_c = np.asarray(angles_c, dtype=np.int32)

    # NumPy 版本
    angles_np = pycm.generate_angles_np(
        bounding_box_size,
        distances,
        bidirectional=bidirectional,
        force2D=force2D,
        force2Ddimension=force2Ddimension,
    ).astype(np.int32)

    # Torch 版本
    angles_torch = pycm.generate_angles_torch(
        bounding_box_size,
        distances,
        bidirectional=bidirectional,
        force2D=force2D,
        force2Ddimension=force2Ddimension,
        dtype=torch.int32,
        device="cpu",
    )
    angles_torch_np = angles_torch.cpu().numpy().astype(np.int32)

    print("shape C / np / torch:", angles_c.shape, angles_np.shape, angles_torch_np.shape)

    same_np = np.array_equal(angles_c, angles_np)
    same_torch = np.array_equal(angles_c, angles_torch_np)

    print("C vs NumPy 全等:", same_np)
    print("C vs Torch 全等:", same_torch)

    if not same_np:
        print("C 与 NumPy 有差异，示例：")
        print("C[0:10]:\n", angles_c[:10])
        print("NP[0:10]:\n", angles_np[:10])

    if not same_torch:
        print("C 与 Torch 有差异，示例：")
        print("C[0:10]:\n", angles_c[:10])
        print("Torch[0:10]:\n", angles_torch_np[:10])


# 举个和你原来代码一致的例子：
check_equal_once(
    bounding_box_size=[5, 5, 5],
    kernel_radius=2,
    force2D=False,
    force2Ddimension=0,
    bidirectional=True,
)

shape C / np / torch: (124, 3) (124, 3) (124, 3)
C vs NumPy 全等: True
C vs Torch 全等: True


In [13]:
import numpy as np
import torch
from radiomics import cMatrices
from radiomics import cmatrices as pycm


# -------------------------
# 单次对比函数
# -------------------------
def compare_case(
    size,
    kernel_radius,
    force2D=False,
    force2Ddim=0,
    bidirectional=True,
):
    size = np.asarray(size, dtype=np.int32)
    distances = np.arange(1, kernel_radius + 1, dtype=np.int32)

    # --- C 扩展 ---
    angles_c = cMatrices.generate_angles(
        size,
        distances,
        bidirectional,
        force2D,
        force2Ddim,
    )
    angles_c = np.asarray(angles_c, dtype=np.int32)

    # --- NumPy 实现 ---
    angles_np = pycm.generate_angles_np(
        size,
        distances,
        bidirectional=bidirectional,
        force2D=force2D,
        force2Ddimension=force2Ddim,
    ).astype(np.int32)

    # --- Torch 实现 ---
    angles_torch = pycm.generate_angles_torch(
        size,
        distances,
        bidirectional=bidirectional,
        force2D=force2D,
        force2Ddimension=force2Ddim,
        dtype=torch.int32,
        device="cpu",
    )
    angles_torch = angles_torch.cpu().numpy().astype(np.int32)

    # --- Check shape ---
    if angles_c.shape != angles_np.shape or angles_c.shape != angles_torch.shape:
        print("❌ shape mismatch")
        print("C:", angles_c.shape)
        print("NP:", angles_np.shape)
        print("Torch:", angles_torch.shape)
        raise AssertionError("shape not match")

    # --- Check values ---
    same_np = np.array_equal(angles_c, angles_np)
    same_torch = np.array_equal(angles_c, angles_torch)

    print(f"Case: size={size.tolist()}, kernel={kernel_radius}, "
          f"force2D={force2D}, dim={force2Ddim}, bid={bidirectional}")

    print("  C vs NumPy  equal:", same_np)
    print("  C vs Torch  equal:", same_torch)

    if not same_np:
        print("  Difference C vs NumPy (first 10 rows):")
        print(angles_c[:10])
        print(angles_np[:10])
        raise AssertionError("C vs NumPy mismatch")

    if not same_torch:
        print("  Difference C vs Torch (first 10 rows):")
        print(angles_c[:10])
        print(angles_torch[:10])
        raise AssertionError("C vs Torch mismatch")

    print("  ✅ OK\n")


# -------------------------
# 批量测试
# -------------------------
print("===== 基础测试 =====")
compare_case([5,5,5], kernel_radius=1)
compare_case([5,5,5], kernel_radius=2)
compare_case([7,7,7], kernel_radius=2)

print("===== force2D 测试 =====")
compare_case([1,7,7], kernel_radius=2, force2D=True, force2Ddim=0)
compare_case([7,1,7], kernel_radius=2, force2D=True, force2Ddim=1)
compare_case([7,7,1], kernel_radius=2, force2D=True, force2Ddim=2)

print("===== 单向测试（bidirectional=False） =====")
compare_case([5,5,5], kernel_radius=1, bidirectional=False)
compare_case([7,7,7], kernel_radius=2, bidirectional=False)

print("===== 2D 测试 =====")
compare_case([10,10], kernel_radius=1)
compare_case([10,10], kernel_radius=3)

print("===== 随机测试 10 组 =====")
rng = np.random.default_rng(0)
for _ in range(10):
    Nd = int(rng.integers(2, 4))  # 2D 或 3D
    size = rng.integers(3, 10, size=Nd)
    kernel = int(rng.integers(1, 4))
    force2D = bool(rng.integers(0, 2))
    force2Ddim = int(rng.integers(0, Nd)) if force2D else 0
    bidirectional = bool(rng.integers(0, 2))

    compare_case(
        size,
        kernel,
        force2D=force2D,
        force2Ddim=force2Ddim,
        bidirectional=bidirectional,
    )

print("🎉 所有测试通过！")

===== 基础测试 =====
Case: size=[5, 5, 5], kernel=1, force2D=False, dim=0, bid=True
  C vs NumPy  equal: True
  C vs Torch  equal: True
  ✅ OK

Case: size=[5, 5, 5], kernel=2, force2D=False, dim=0, bid=True
  C vs NumPy  equal: True
  C vs Torch  equal: True
  ✅ OK

Case: size=[7, 7, 7], kernel=2, force2D=False, dim=0, bid=True
  C vs NumPy  equal: True
  C vs Torch  equal: True
  ✅ OK

===== force2D 测试 =====
Case: size=[1, 7, 7], kernel=2, force2D=True, dim=0, bid=True
  C vs NumPy  equal: True
  C vs Torch  equal: True
  ✅ OK

Case: size=[7, 1, 7], kernel=2, force2D=True, dim=1, bid=True
  C vs NumPy  equal: True
  C vs Torch  equal: True
  ✅ OK

Case: size=[7, 7, 1], kernel=2, force2D=True, dim=2, bid=True
  C vs NumPy  equal: True
  C vs Torch  equal: True
  ✅ OK

===== 单向测试（bidirectional=False） =====
Case: size=[5, 5, 5], kernel=1, force2D=False, dim=0, bid=False
  C vs NumPy  equal: True
  C vs Torch  equal: True
  ✅ OK

Case: size=[7, 7, 7], kernel=2, force2D=False, dim=0, bid=False

In [16]:
def benchmark_one_case(
    size,
    kernel_radius,
    force2D=False,
    force2Ddim=0,
    bidirectional=True,
    n_runs=50,
):
    size = np.asarray(size, dtype=np.int32)
    distances = np.arange(1, kernel_radius + 1, dtype=np.int32)

    print("=" * 80)
    print(f"Case: size={size.tolist()}, kernel_radius={kernel_radius}, "
          f"force2D={force2D}, force2Ddim={force2Ddim}, bidirectional={bidirectional}")
    print(f"Runs per impl: {n_runs}")

    # 先跑一遍做 warmup，避免第一次调用的开销（import、JIT 等）影响
    _ = cMatrices.generate_angles(size, distances, bidirectional, force2D, force2Ddim)
    _ = pycm.generate_angles_np(size, distances, bidirectional, force2D, force2Ddim)
    _ = pycm.generate_angles_torch(
        size,
        distances,
        bidirectional=bidirectional,
        force2D=force2D,
        force2Ddimension=force2Ddim,
        dtype=torch.int32,
        device="cpu",
    )

    # --------- C 实现 ----------
    t0 = time.perf_counter()
    for _ in range(n_runs):
        _ = cMatrices.generate_angles(size, distances, bidirectional, force2D, force2Ddim)
    t_c = (time.perf_counter() - t0) / n_runs

    # --------- NumPy 实现 ----------
    t0 = time.perf_counter()
    for _ in range(n_runs):
        _ = pycm.generate_angles_np(size, distances, bidirectional, force2D, force2Ddim)
    t_np = (time.perf_counter() - t0) / n_runs

    # --------- Torch 实现（CPU） ----------
    size_torch = torch.as_tensor(size, dtype=torch.int32, device="cpu")
    dist_torch = torch.as_tensor(distances, dtype=torch.int32, device="cpu")

    t0 = time.perf_counter()
    for _ in range(n_runs):
        _ = pycm.generate_angles_torch(
            size_torch,
            dist_torch,
            bidirectional=bidirectional,
            force2D=force2D,
            force2Ddimension=force2Ddim,
            dtype=torch.int32,
            device="cpu",
        )
    t_torch = (time.perf_counter() - t0) / n_runs

    print(f"  C      avg time: {t_c*1e3:8.3f} ms")
    print(f"  NumPy  avg time: {t_np*1e3:8.3f} ms  (x{t_np/t_c:5.2f} vs C)")
    print(f"  Torch  avg time: {t_torch*1e3:8.3f} ms  (x{t_torch/t_c:5.2f} vs C)")

In [17]:
def benchmark_one_case(
    size,
    kernel_radius,
    force2D=False,
    force2Ddim=0,
    bidirectional=True,
    n_runs=50,
):
    size = np.asarray(size, dtype=np.int32)
    distances = np.arange(1, kernel_radius + 1, dtype=np.int32)

    print("=" * 80)
    print(f"Case: size={size.tolist()}, kernel_radius={kernel_radius}, "
          f"force2D={force2D}, force2Ddim={force2Ddim}, bidirectional={bidirectional}")
    print(f"Runs per impl: {n_runs}")

    # 先跑一遍做 warmup，避免第一次调用的开销（import、JIT 等）影响
    _ = cMatrices.generate_angles(size, distances, bidirectional, force2D, force2Ddim)
    _ = pycm.generate_angles_np(size, distances, bidirectional, force2D, force2Ddim)
    _ = pycm.generate_angles_torch(
        size,
        distances,
        bidirectional=bidirectional,
        force2D=force2D,
        force2Ddimension=force2Ddim,
        dtype=torch.int32,
        device="cpu",
    )

    # --------- C 实现 ----------
    t0 = time.perf_counter()
    for _ in range(n_runs):
        _ = cMatrices.generate_angles(size, distances, bidirectional, force2D, force2Ddim)
    t_c = (time.perf_counter() - t0) / n_runs

    # --------- NumPy 实现 ----------
    t0 = time.perf_counter()
    for _ in range(n_runs):
        _ = pycm.generate_angles_np(size, distances, bidirectional, force2D, force2Ddim)
    t_np = (time.perf_counter() - t0) / n_runs

    # --------- Torch 实现（CPU） ----------
    size_torch = torch.as_tensor(size, dtype=torch.int32, device="cpu")
    dist_torch = torch.as_tensor(distances, dtype=torch.int32, device="cpu")

    t0 = time.perf_counter()
    for _ in range(n_runs):
        _ = pycm.generate_angles_torch(
            size_torch,
            dist_torch,
            bidirectional=bidirectional,
            force2D=force2D,
            force2Ddimension=force2Ddim,
            dtype=torch.int32,
            device="cpu",
        )
    t_torch = (time.perf_counter() - t0) / n_runs

    print(f"  C      avg time: {t_c*1e3:8.3f} ms")
    print(f"  NumPy  avg time: {t_np*1e3:8.3f} ms  (x{t_np/t_c:5.2f} vs C)")
    print(f"  Torch  avg time: {t_torch*1e3:8.3f} ms  (x{t_torch/t_c:5.2f} vs C)")

In [19]:
# 大一点的 3D
benchmark_one_case(
    size=[21, 21, 21],
    kernel_radius=3,
    n_runs=10,
)

# 再大一点 2D
benchmark_one_case(
    size=[51, 51],
    kernel_radius=7,
    n_runs=10,
)

Case: size=[21, 21, 21], kernel_radius=3, force2D=False, force2Ddim=0, bidirectional=True
Runs per impl: 10
  C      avg time:    0.003 ms
  NumPy  avg time:    0.281 ms  (x82.38 vs C)
  Torch  avg time:    2.519 ms  (x738.14 vs C)
Case: size=[51, 51], kernel_radius=7, force2D=False, force2Ddim=0, bidirectional=True
Runs per impl: 10
  C      avg time:    0.002 ms
  NumPy  avg time:    0.108 ms  (x64.05 vs C)
  Torch  avg time:    0.953 ms  (x567.35 vs C)


In [21]:
import time
import torch
import numpy as np
from radiomics import cMatrices
from radiomics import cmatrices as pycm


def benchmark_torch_gpu(
    size,
    kernel_radius,
    force2D=False,
    force2Ddim=0,
    bidirectional=True,
    n_runs=50,
):
    size = np.asarray(size, dtype=np.int32)
    distances = np.arange(1, kernel_radius + 1, dtype=np.int32)

    size_t = torch.as_tensor(size, dtype=torch.int32, device="cuda")
    dist_t = torch.as_tensor(distances, dtype=torch.int32, device="cuda")

    # warmup（必须）
    for _ in range(5):
        _ = pycm.generate_angles_torch(
            size_t,
            dist_t,
            bidirectional=bidirectional,
            force2D=force2D,
            force2Ddimension=force2Ddim,
            dtype=torch.int32,
            device="cuda",
        )
        torch.cuda.synchronize()

    # benchmark
    t0 = time.perf_counter()
    for _ in range(n_runs):
        _ = pycm.generate_angles_torch(
            size_t,
            dist_t,
            bidirectional=bidirectional,
            force2D=force2D,
            force2Ddimension=force2Ddim,
            dtype=torch.int32,
            device="cuda",
        )
        torch.cuda.synchronize()
    t_gpu = (time.perf_counter() - t0) / n_runs

    print(f"GPU avg time: {t_gpu*1000:.4f} ms")
    return t_gpu

In [22]:
def compare_cpu_vs_gpu(
    size,
    kernel_radius,
    force2D=False,
    force2Ddim=0,
    bidirectional=True,
):
    size = np.asarray(size, dtype=np.int32)
    distances = np.arange(1, kernel_radius + 1, dtype=np.int32)

    # CPU
    cpu_out = pycm.generate_angles_torch(
        size,
        distances,
        bidirectional=bidirectional,
        force2D=force2D,
        force2Ddimension=force2Ddim,
        dtype=torch.int32,
        device="cpu",
    ).cpu().numpy()

    # GPU
    gpu_out = pycm.generate_angles_torch(
        torch.as_tensor(size, dtype=torch.int32, device="cuda"),
        torch.as_tensor(distances, dtype=torch.int32, device="cuda"),
        bidirectional=bidirectional,
        force2D=force2D,
        force2Ddimension=force2Ddim,
        dtype=torch.int32,
        device="cuda",
    ).cpu().numpy()

    print("Same:", np.array_equal(cpu_out, gpu_out))
    if not np.array_equal(cpu_out, gpu_out):
        print("CPU sample:\n", cpu_out[:10])
        print("GPU sample:\n", gpu_out[:10])

In [23]:
compare_cpu_vs_gpu([5,5,5], 2)

AssertionError: Torch not compiled with CUDA enabled